# Make Matching Pairs with Embeddings

In [5]:
import pickle
import torch
from pathlib import Path
from lib.glove import GloveEmbeddings

In [4]:
glove_pkl_path = Path.home()/"data"/"glove.pkl"
g = pickle.load(open(glove_pkl_path, "rb"))

## Core Idea

Here, we do a matching pair like this:

* france, paris

Then if we ask for a matching similar work for "italy", it should output "rome" or similar.

In [56]:
w1 = "italy"
w2 = "rome"
w3 = "france"

# let's find the direction
dir_1 = g.make(w1) - g.make(w2)
dir_1.shape, dir_1

(torch.Size([1, 50]),
 tensor([[ 0.0376, -1.5225, -0.0436,  0.9482, -0.0748,  1.2664, -0.0514, -0.2955,
           0.3791, -0.0058,  0.8065, -0.5466, -0.0591,  0.2315,  0.1583, -0.2131,
           0.5206,  0.1262, -0.0181, -0.5960, -0.5613, -0.3547, -0.1362,  0.5711,
           0.0073, -0.0623,  0.5180, -0.0079,  0.0105,  0.3035,  0.8375,  0.5943,
           0.1780,  0.8234,  0.0449, -0.0925, -0.7842,  0.4940, -0.3009, -1.2704,
          -0.0706,  0.1819,  0.7480, -0.9892,  0.1189,  0.3937,  0.3016,  0.2313,
           0.5588, -0.0074]]))

In [57]:
# let's find the direction of w3 with all other embeddings
dir_2 = g.make(w3) - g.embs
dir_2.shape

torch.Size([400001, 50])

In [59]:
# let's get similarities
similarities = torch.cosine_similarity(dir_1, dir_2)
similarities

tensor([0.3063, 0.2554, 0.3136,  ..., 0.2409, 0.3693, 0.3693])

In [60]:
# let do a argmax & find the word
g.words[torch.argmax(similarities)]

'paris'

**That's exactly we are looking for**

Let's make a function for this

In [61]:
def find_matching(w1, w2, w3):
    dir_1 = g.make(w1) - g.make(w2)
    dir_2 = g.make(w3) - g.embs
    
    similarities = torch.cosine_similarity(dir_1, dir_2)
    max_index = torch.argmax(similarities)
    
    return g.words[max_index]

In [62]:
find_matching("paris", "france", "rome")

'italy'

In [64]:
find_matching("paris", "france", "delhi")

'pakistan'

In [67]:
find_matching("italy", "rome", "russia")

'moscow'

In [69]:
find_matching("king", "man", "queen")

'woman'

In [74]:
find_matching("man", "engineer", "woman")

'engineer'

In [75]:
find_matching("man", "king", "woman")

'king'

In [92]:
find_matching("india", "ruppees", "usa")

'rayos'

See. There are some matching values here & there :)

**But it's a good one.**